<a href="https://colab.research.google.com/github/ryan9453/0602/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
from urllib.request import urlretrieve
url = 'https://github.com/ryan9453/0602/raw/main/titanic/train.csv'
urlretrieve(url, 'train.csv')
url = 'https://github.com/ryan9453/0602/raw/main/titanic/test.csv'
urlretrieve(url, 'test.csv')

('test.csv', <http.client.HTTPMessage at 0x7fc8ebbb4750>)

In [58]:
import pandas as pd
train_df = pd.read_csv('train.csv', encoding = 'utf-8')
test_df = pd.read_csv('test.csv', encoding = 'utf-8')

In [59]:
data = pd.concat([train_df, test_df], ignore_index = True)
data = data.drop(['PassengerId', 'Survived'], axis = 1)

test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [60]:
na = data.isna().sum()
# na

# Series[帶入跟你的資料筆數一樣多的 True/False list]
# na = data.isna().sum()
na[na > 0].sort_values(ascending = False)


Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [61]:
# 補類別(最常出現)
def cabin_head(s):
    if not pd.isna(s):
        return s[0]
data['Cabin'] = data['Cabin'].apply(cabin_head)

In [62]:
dic = data['Ticket'].value_counts()
data['Ticket'] = data['Ticket'].apply(lambda t:dic[t])

In [64]:
# 補缺失值 (類別: 最常出現) [!如果一堆測試資料，不要重算，直接補S]
most = data['Embarked'].value_counts().idxmax()
data['Embarked'] = data['Embarked'].fillna(most)


In [65]:
# 補缺失(數值: 中位數)
med = data.median().drop(['Pclass'])
data = data.fillna(med)
na = data.isna().sum()
na[na > 0].sort_values(ascending = False)

Cabin    1014
dtype: int64

In [67]:
def name_convert(s):
    s = s.split(',')[-1].split('.')[0]
    s = s.strip() # 去除多餘空格
    return s
counts = data['Name'].apply(name_convert).value_counts()
whitelist = counts[counts > 50].index

def name_convert(s):
    s = s.split(',')[-1].split('.')[0]
    s = s.strip() # 去除多餘空格
    if s in whitelist:
        return s
    else:
        return None
data['Name'] = data['Name'].apply(name_convert)

In [69]:
data = pd.get_dummies(data)
data = pd.get_dummies(data, columns = ['Pclass'])
data

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [70]:
# 模型建立完，後續的嘗試，只是老師嘗試過，所以直接做
data['Family'] = data['SibSp'] + data['Parch']
data

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Family
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
